In [1]:
# This is a Python 3 environment

# Base level imports for data science work
import numpy as np 
import pandas as pd
import re,string,unicodedata
import os
from os import path
from json import decoder
import glob, os, json
import re

# Visualization Libs
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# NLP Libs
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
from keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# Additional Libs
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet

# Deep Learning Libs
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [2]:
df = pd.read_csv("https://chruezfalsch.ch/df_FANG-COVID_cleaned.csv")
del df['Unnamed: 0'] 

In [3]:
!wget https://chruezfalsch.ch/ba_model_1.joblib
!wget https://chruezfalsch.ch/ba_model_1_cv.joblib
!wget https://chruezfalsch.ch/ba_model_1_tfidf.joblib


--2022-05-15 10:02:05--  https://chruezfalsch.ch/ba_model_1.joblib
Resolving chruezfalsch.ch (chruezfalsch.ch)... 185.151.30.135, 2a07:7800::135
Connecting to chruezfalsch.ch (chruezfalsch.ch)|185.151.30.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38964509 (37M)
Saving to: ‘ba_model_1.joblib’

ba_model_1.joblib   100%[===================>]  37.16M   575KB/s    in 69s     

2022-05-15 10:03:15 (555 KB/s) - ‘ba_model_1.joblib’ saved [38964509/38964509]

--2022-05-15 10:03:15--  https://chruezfalsch.ch/ba_model_1_cv.joblib
Resolving chruezfalsch.ch (chruezfalsch.ch)... 185.151.30.135, 2a07:7800::135
Connecting to chruezfalsch.ch (chruezfalsch.ch)|185.151.30.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42952136 (41M)
Saving to: ‘ba_model_1_cv.joblib’

ba_model_1_cv.jobli 100%[===================>]  40.96M  6.88MB/s    in 8.7s    

2022-05-15 10:03:25 (4.73 MB/s) - ‘ba_model_1_cv.joblib’ saved [42952136/42952136]

--2022-05-

In [4]:
import pickle
from joblib import dump, load
mnb = load('ba_model_1.joblib')
cv = pickle.load(open('ba_model_1_cv.joblib','rb'))
tfidf = pickle.load(open('ba_model_1_tfidf.joblib','rb'))

#Model

In [5]:
X_test = df.text
y_test = df.fake

# Transform the test dataset
cv_test=cv.transform(X_test)
print('Bag of words Test:',cv_test.shape)

Bag of words Test: (28589, 1217620)


In [6]:
# Transform the test dataset
tfidf_test=tfidf.transform(X_test)
print('TF-IDF Test:',tfidf_test.shape)

TF-IDF Test: (28589, 1217620)


In [7]:
# Predicting the model for bag of words
mnb_cv_predict=mnb.predict(cv_test)

# Predicting the model for tf-idf features
mnb_tfidf_predict=mnb.predict(tfidf_test)

In [8]:
# Check the accuracy score for bag of words
mnb_cv_score=accuracy_score(y_test, mnb_cv_predict)
print("Naive Bayes Bag of words accuracy score:", mnb_cv_score)

# Check the accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(y_test,mnb_tfidf_predict)
print("Naive Bayes TF-IDF accuracy score:", mnb_tfidf_score)

Naive Bayes Bag of words accuracy score: 0.7376963167651894
Naive Bayes TF-IDF accuracy score: 0.7066703977054112


In [9]:
mnb_cv_report = classification_report(y_test, mnb_cv_predict, target_names = ['0','1'])
print(mnb_cv_report)
mnb_tfidf_report = classification_report(y_test, mnb_tfidf_predict, target_names = ['0','1'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

           0       0.86      0.76      0.81     20810
           1       0.51      0.67      0.58      7779

    accuracy                           0.74     28589
   macro avg       0.69      0.72      0.69     28589
weighted avg       0.77      0.74      0.75     28589

              precision    recall  f1-score   support

           0       0.88      0.69      0.77     20810
           1       0.48      0.75      0.58      7779

    accuracy                           0.71     28589
   macro avg       0.68      0.72      0.68     28589
weighted avg       0.77      0.71      0.72     28589



#Model (ausbalanciert)

In [12]:
dfb=df[df.fake == 0].sample(5000)
dfb = dfb.append(df[df.fake == 1].sample(5000))
X_test = dfb.text
y_test = dfb.fake

# Transform the test dataset
cv_test=cv.transform(X_test)
print('Bag of words Test:',cv_test.shape)

Bag of words Test: (10000, 1217620)


In [13]:
# Transform the test dataset
tfidf_test=tfidf.transform(X_test)
print('TF-IDF Test:',tfidf_test.shape)

TF-IDF Test: (10000, 1217620)


In [14]:
# Predicting the model for bag of words
mnb_cv_predict=mnb.predict(cv_test)

# Predicting the model for tf-idf features
mnb_tfidf_predict=mnb.predict(tfidf_test)

In [15]:
# Check the accuracy score for bag of words
mnb_cv_score=accuracy_score(y_test, mnb_cv_predict)
print("Naive Bayes Bag of words accuracy score:", mnb_cv_score)

# Check the accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(y_test,mnb_tfidf_predict)
print("Naive Bayes TF-IDF accuracy score:", mnb_tfidf_score)

Naive Bayes Bag of words accuracy score: 0.7175
Naive Bayes TF-IDF accuracy score: 0.7247


In [16]:
mnb_cv_report = classification_report(y_test, mnb_cv_predict, target_names = ['0','1'])
print(mnb_cv_report)
mnb_tfidf_report = classification_report(y_test, mnb_tfidf_predict, target_names = ['0','1'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

           0       0.70      0.77      0.73      5000
           1       0.74      0.67      0.70      5000

    accuracy                           0.72     10000
   macro avg       0.72      0.72      0.72     10000
weighted avg       0.72      0.72      0.72     10000

              precision    recall  f1-score   support

           0       0.74      0.70      0.72      5000
           1       0.71      0.75      0.73      5000

    accuracy                           0.72     10000
   macro avg       0.73      0.72      0.72     10000
weighted avg       0.73      0.72      0.72     10000

